In [ ]:
import numpy as np
import numpy.polynomial as P
import scipy as sp
from matplotlib import pyplot as plt
from tqdm import tqdm
import Algo2
import Algo1
import ULA
from sklearn.preprocessing import PolynomialFeatures
from joblib import Parallel, delayed
import ZVnbrosse

In [ ]:
import importlib
importlib.reload(Algo2)

In [ ]:
def f_grad(x):
    a = 0.0
    return x-a+2*a/(1 + np.exp(2* (x * a)))

def f(x):
    a = 1 / np.sqrt(2)
    return 1/2 * (x-a)**2 - np.log(1 + np.exp(-2 * x * a))

def set_func(x):
    #function of interest to compute averages
    return x

In [ ]:
#for normal distribution
a = 0.0
sigma = 1.0

def f_grad(x):
    
    

Approximation results: use $Q_{l-p}$ to approximate family of $Q_{l,p}$ relying on approximate stationarity of the chain.

In [ ]:
#generate samples from mixture of normals
N_burn = 10000
N_train = 10000
gamma = 0.2
N_traj_train = 1
X_train = np.zeros((N_traj_train,N_train),dtype = float)

for j in range(N_traj_train):
    np.random.seed(42+j)
    x0 = np.random.randn()
    x_cur = x0
    #burn-in
    for i in range(N_burn):
        x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
    #training sample
    for i in range(N_train):
        X_train[j,i] = x_cur
        x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
X_last = X_train[0,-1]

In [ ]:
print(np.mean(X_train[0,:]))

Optimize coefficients

In [ ]:
#degree
max_deg = 2
#lag order
lag = 10
#polynomial coefficients
coefs_poly = np.zeros((lag,max_deg+1),dtype = float)

for i in range(lag):
    y = set_func(X_train[0,(i+1):])
    x = X_train[0,:-(i+1)]
    res = P.polynomial.polyfit(x,y,max_deg)
    coefs_poly[i,:] = res

In [ ]:
print(coefs_poly[0,:])

Test our regressors

In [ ]:
cur_lag = 5
N_pts = 100
plt.figure(figsize=(10, 10))
plt.title("Testing regression model",fontsize=20)
plt.plot(X_train[0,cur_lag:N_pts],color='r',label='true function')
plt.plot(P.polynomial.polyval(X_train[0,:N_pts-cur_lag],coefs_poly[cur_lag-1,:]),color='g',label = 'approximation')
plt.legend(loc = 'lower right',fontsize = 16)
plt.show()

In [ ]:
sp.special.factorial2(21, exact=False)
moments_stand_norm = np.zeros(2*max_deg+1,dtype = float)
for i in range(len(moments_stand_norm)):
    moments_stand_norm[i] = sp.special.factorial2(i+1, exact=False)
#eliminate odd
moments_stand_norm[1::2] = 0
print(moments_stand_norm)

In [ ]:
N_burn = 10000
N_test = 2000 #size of test part
N_min = 1000 #minimal number of observations to compute \pi_N
gamma = 0.2
X_test = np.zeros(N_test,dtype = float)
Noise = np.zeros_like(X_test)
N_traj = 10

test_stat_vanilla = np.zeros((N_traj,N_test),dtype = float)
test_stat_vr = np.zeros((N_traj,N_test),dtype = float)

In [ ]:
for ind in range(N_traj):
    np.random.seed(1453 + ind)
    #x0 = np.random.randn()
    #x_cur = x0
    #burn-in
    #for i in range(N_burn):
        #x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
    #training sample
    x_cur = X_last
    for i in range(N_test):
        Noise[i] = np.random.randn()
        x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*Noise[i]
        X_test[i] = x_cur
    #compute polynomials at Z_l
    poly_vals = np.zeros((max_deg+1,N_test),dtype = float)
    for k in range(max_deg+1):
        c = np.zeros(max_deg+1)
        c[k] = 1
        poly_vals[k,:] = P.hermite_e.hermeval(Noise,c)/np.sqrt((np.sqrt(2*np.pi)*sp.special.factorial(k)))
    f_vals_vanilla = set_func(X_test)
    cvfs = np.zeros_like(f_vals_vanilla)
    for i in range(100,len(cvfs)):
        #start computing a_{p-l} coefficients
        num_poly = min(lag,i)
        a_vals = np.zeros((num_poly,max_deg+1),dtype = float)
        for npol in range(num_poly):#for a fixed lag Q function
            #compute \hat{a} with fixed lag
            x = X_test[i-2-npol]#should be -2 here?
            a_cur = np.zeros(max_deg+1,dtype=float)
            for m in range(len(coefs_poly[0])):
                poly_vspom = np.zeros(max_deg+1,dtype=float)
                for u in range(m+1):
                    poly_vspom[u] = ((x-gamma*f_grad(x))**(m-u))*((np.sqrt(2*gamma))**u)*sp.special.binom(m,u)
                a_cur = P.polynomial.polyadd(a_cur,coefs_poly[npol,m]*poly_vspom) 
            for k in range(max_deg+1):
                c = np.zeros(max_deg+1)
                c[k] = 1
                herm_coef = P.hermite_e.herme2poly(c)
                #normalize now
                herm_coef = herm_coef / np.sqrt((np.sqrt(2*np.pi)*sp.special.factorial(k)))
                integr_coefs = P.polynomial.polymul(herm_coef,a_cur)
                #Note that a_vals are stored in reversed order
                a_vals[-(npol+1),k] = np.dot(integr_coefs,moments_stand_norm[:len(integr_coefs)])
            #OK, now I have coefficients of the polynomial, and I need to integrate it w.r.t. Gaussian measure
        cvfs[i] = np.sum(a_vals*(poly_vals[:,i-num_poly+1:i+1].T))
        #cvfs[i] = np.sum(np.mean(a_vals,axis = 0))
        if (i%100 == 0):
            print("100 observations proceeded")
        #save results
        test_stat_vanilla[ind,i] = np.mean(f_vals_vanilla[:i])
        test_stat_vr[ind,i] = test_stat_vanilla[ind,i] - cvfs[i]/i

In [ ]:
print(test_stat_vanilla[1,N_min:N_min+100])
print(test_stat_vr[1,N_min:N_min+100])

In [ ]:
vars_vanilla = np.var(test_stat_vanilla,axis = 0)
vars_adj = np.var(test_stat_vr,axis = 0)
print(vars_vanilla[N_min:N_min+10])
print(vars_adj[N_min:N_min+10])
print(np.mean(vars_adj[N_min:]/vars_vanilla[N_min:]))